### Sentiment classification with ChoiceNet

In [1]:
import os
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
from util import load_directory_data,load_dataset,download_and_load_datasets,gpusession,random_flip_target
from cn_cls_class import cn_cls_class
print ("Packages loaded.")

Packages loaded.


### Get feature

In [2]:
dataset_name = 'movie_review'

In [3]:
savename = 'data/' + dataset_name + '.npz'
FORCE_RELOAD = 0 
if os.path.isfile(savename) & (~FORCE_RELOAD): # load if the file exists,
    # Load 
    l = np.load(savename)    
    x_train = l['x_train']
    x_test = l['x_test']
    t_train = l['t_train']
    t_test = l['t_test']
    print ("[%s] Loaded. Size is [%.4f]MB" % (savename,os.path.getsize(savename)/1000./1000.))
else: # otherwise, make the data
    # Download dataset
    tf.logging.set_verbosity(tf.logging.ERROR) # Reduce logging output.
    train_df, test_df = download_and_load_datasets()
    # Check dataset
    for i in range(2):
        print (i,':',train_df['sentence'][i],'\n')
    for i in range(2):
        print (i,':',train_df['polarity'][i])
    embed_module = hub.Module("https://tfhub.dev/google/nnlm-en-dim128/1")
    embed_train = embed_module(tf.reshape(train_df["sentence"], shape=[-1]))
    embed_test = embed_module(tf.reshape(test_df["sentence"], shape=[-1]))
    with tf.train.MonitoredTrainingSession(is_chief=True) as sess:
        x_train = sess.run(embed_train)
        x_test = sess.run(embed_test)
    n_train,n_test = np.shape(x_train)[0],np.shape(x_test)[0]
    t_train,t_test = np.zeros((n_train,2)),np.zeros((n_test,2))
    for i in range(n_train):
        t_train[i,train_df['polarity'][i]] = 1
    for i in range(n_test):
        t_test[i,test_df['polarity'][i]] = 1    
    print("Shapes of 'x_train' and 'x_test' are %s and %s."%
          (x_train.shape,x_test.shape)) # (result: (1, 128))    
    print("Shapes of 't_train' and 't_test' are %s and %s."%
          (t_train.shape,t_test.shape)) # (result: (1, 128))    
    # Save 
    np.savez(savename,x_train=x_train,x_test=x_test,t_train=t_train,t_test=t_test)
    print ("[%s] Saved. Size is [%.4f]MB" % (savename,os.path.getsize(savename)/1000./1000.))

[data/movie_review.npz] Loaded. Size is [26.4009]MB


### Run

In [4]:
# Add perturbation to t_label
t_train_shuffle = random_flip_target(t_train,_rate=0.3,_seed=0)

In [6]:
tf.reset_default_graph(); tf.set_random_seed(0)
CN = cn_cls_class(_name='cn_cls_sentiment',_x_dim=128,_t_dim=2,_h_dims=[128,128],
                  _k_mix=5,_rho_ref_train=0.95,_tau_inv=1e-4,_pi1_bias=0.0,
                  _log_sigma_z_val=-2,_logsumexp_coef=0.1,_kl_reg_coef=0.1,
                  _actv=tf.nn.relu,_bn=slim.batch_norm,_l2_reg_coef=1e-6,_momentum=0.5,
                  _USE_SGD=False,_USE_MIXUP=False,_GPU_ID=0,_VERBOSE=True)
sess = gpusession();sess.run(tf.global_variables_initializer()) 
CN.train(sess,_x_train=x_train,_t_train=t_train_shuffle,_x_test=x_test,_t_test=t_test,
              _max_epoch=20,_batch_size=128,_lr=1e-5,_kp=0.9,
              _LR_SCHEDULE=0,_PRINT_EVERY=10,_VERBOSE_TRAIN=True)

==== Global Variables ====
 [00] Name:[cn_cls_sentiment/lin0/weights:0] Shape:[[128, 128]]
 [01] Name:[cn_cls_sentiment/lin0/BatchNorm/beta:0] Shape:[[128]]
 [02] Name:[cn_cls_sentiment/lin0/BatchNorm/moving_mean:0] Shape:[[128]]
 [03] Name:[cn_cls_sentiment/lin0/BatchNorm/moving_variance:0] Shape:[[128]]
 [04] Name:[cn_cls_sentiment/lin1/weights:0] Shape:[[128, 128]]
 [05] Name:[cn_cls_sentiment/lin1/BatchNorm/beta:0] Shape:[[128]]
 [06] Name:[cn_cls_sentiment/lin1/BatchNorm/moving_mean:0] Shape:[[128]]
 [07] Name:[cn_cls_sentiment/lin1/BatchNorm/moving_variance:0] Shape:[[128]]
 [08] Name:[cn_cls_sentiment/rho_raw/weights:0] Shape:[[128, 5]]
 [09] Name:[cn_cls_sentiment/rho_raw/BatchNorm/beta:0] Shape:[[5]]
 [10] Name:[cn_cls_sentiment/rho_raw/BatchNorm/moving_mean:0] Shape:[[5]]
 [11] Name:[cn_cls_sentiment/rho_raw/BatchNorm/moving_variance:0] Shape:[[5]]
 [12] Name:[cn_cls_sentiment/muW:0] Shape:[[128, 2]]
 [13] Name:[cn_cls_sentiment/logSigmaW:0] Shape:[[128, 2]]
 [14] Name:[cn_cl